# Evaluating a PyCon Assistant Copilot using Azure AI Evaluation SDK

This notebook demonstrates how to evaluate a custom copilot that acts as a PyCon conference assistant using the Azure AI Evaluation SDK. We'll set up evaluation datasets, define custom evaluation criteria, and analyze the assistant's responses for quality, relevance, and helpfulness in the context of Python conference information.

## Installation

First, let's install the required packages:

In [1]:
# Install the Azure AI Evaluation SDK and other required packages
!pip install azure-ai-evaluation azure-identity openai

  Using cached azure_ai_evaluation-1.5.0-py3-none-any.whl.metadata (35 kB)
  Using cached azure_identity-1.21.0-py3-none-any.whl.metadata (81 kB)
  Using cached azure_identity-1.21.0-py3-none-any.whl.metadata (81 kB)
  Using cached promptflow_devkit-1.17.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached promptflow_devkit-1.17.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached promptflow_core-1.17.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached promptflow_core-1.17.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached azure_core-1.33.0-py3-none-any.whl.metadata (42 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached azure_storage_blob-12.25.1-py3-none-any.whl.metadata (26 kB)
  Using cached azure_storage_

## Setup and Configuration

Let's set up our environment variables and authentication for Azure services:

In [3]:
import os
import json
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI

# Set up authentication
credential = DefaultAzureCredential()

# Initialize Azure AI project and Azure OpenAI connection
# Replace these with your actual values or set as environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

# Model configuration for evaluators that use LLMs
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION", "2023-12-01-preview"),
    "deployment_name": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
}

# Initialize Azure OpenAI client for our assistant simulation
client = AzureOpenAI(
    azure_endpoint=model_config["azure_endpoint"],
    api_key=model_config["api_key"],
    api_version=model_config["api_version"]
)

OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

## Define PyCon Assistant System Prompt

Here we'll define the system prompt that turns a general LLM into a PyCon conference assistant:

In [ ]:
PYCON_ASSISTANT_SYSTEM_PROMPT = """
You are a helpful PyCon conference assistant. Your primary role is to assist conference attendees 
with information about the conference schedule, talks, workshops, speakers, and logistics. 

You have the following knowledge about PyCon 2025:
- The conference is happening May 15-23, 2025 in Pittsburgh, PA
- Main conference days are May 17-19, with tutorials on May 15-16 and sprints on May 20-23
- Registration opens December 15, 2024 with early bird pricing until February 1, 2025
- The venue is the David L. Lawrence Convention Center in downtown Pittsburgh
- Keynote speakers include Guido van Rossum, Dr. Russell Keith-Magee, and Dr. Łukasz Langa
- There are 95 scheduled talks across 5 parallel tracks, 30 tutorials, and 15 sponsored workshops
- Conference themes include: AI/ML, Web Development, Data Science, Python Core, and Python in Education

Important policies include:
- All attendees must follow the conference Code of Conduct
- Refunds available up to 30 days before the conference with a 10% processing fee
- Limited financial aid is available through an application process that closes February 28, 2025

Answer questions helpfully, accurately, and concisely. If you don't know an answer, acknowledge that 
and suggest where the attendee might find the information (e.g., the registration desk, conference website, etc.).
"""

## Create a PyCon Assistant Simulation

We'll create a function that simulates our PyCon assistant by using the Azure OpenAI service with our system prompt:

In [ ]:
def pycon_assistant(query):
    """Simulate a PyCon assistant by using Azure OpenAI with a specific system prompt"""
    try:
        response = client.chat.completions.create(
            model=model_config["deployment_name"],
            messages=[
                {"role": "system", "content": PYCON_ASSISTANT_SYSTEM_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.3,  # Lower temperature for more consistent answers
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in pycon_assistant: {e}")
        return f"Sorry, I encountered an error: {str(e)}"

## Create PyCon-Specific Test Dataset

Now, let's create a dataset of PyCon-specific questions that we'll use to evaluate our assistant:

In [ ]:
# Create test dataset with questions, relevant context, and expected answer patterns
pycon_test_data = [
    {
        "query": "When is PyCon 2025 happening?",
        "context": "The conference is happening May 15-23, 2025 in Pittsburgh, PA.",
        "expected_pattern": "May 15-23, 2025",
        "category": "logistics"
    },
    {
        "query": "Who are the keynote speakers?",
        "context": "Keynote speakers include Guido van Rossum, Dr. Russell Keith-Magee, and Dr. Łukasz Langa.",
        "expected_pattern": "Guido van Rossum|Russell Keith-Magee|Łukasz Langa",
        "category": "speakers"
    },
    {
        "query": "How much does registration cost?",
        "context": "Registration opens December 15, 2024 with early bird pricing until February 1, 2025.",
        "expected_pattern": "",  # No specific pricing was provided in system prompt
        "category": "registration"
    },
    {
        "query": "Do you have a code of conduct?",
        "context": "All attendees must follow the conference Code of Conduct.",
        "expected_pattern": "Code of Conduct",
        "category": "policy"
    },
    {
        "query": "Can I get a refund if I can't attend?",
        "context": "Refunds available up to 30 days before the conference with a 10% processing fee.",
        "expected_pattern": "30 days|10%",
        "category": "policy"
    },
    {
        "query": "How many tracks will the conference have?",
        "context": "There are 95 scheduled talks across 5 parallel tracks, 30 tutorials, and 15 sponsored workshops.",
        "expected_pattern": "5 parallel tracks|5 tracks",
        "category": "schedule"
    },
    {
        "query": "What topics will be covered at PyCon 2025?",
        "context": "Conference themes include: AI/ML, Web Development, Data Science, Python Core, and Python in Education.",
        "expected_pattern": "AI/ML|Web Development|Data Science|Python Core|Python in Education",
        "category": "content"
    },
    {
        "query": "Is there financial aid available?",
        "context": "Limited financial aid is available through an application process that closes February 28, 2025.",
        "expected_pattern": "financial aid|application|February 28",
        "category": "logistics"
    },
    {
        "query": "Where exactly is the conference venue located?",
        "context": "The venue is the David L. Lawrence Convention Center in downtown Pittsburgh.",
        "expected_pattern": "David L. Lawrence Convention Center|downtown Pittsburgh",
        "category": "logistics"
    },
    {
        "query": "When do the tutorial sessions happen?",
        "context": "Main conference days are May 17-19, with tutorials on May 15-16 and sprints on May 20-23.",
        "expected_pattern": "May 15-16",
        "category": "schedule"
    }
]

# Generate responses from our PyCon assistant
for item in pycon_test_data:
    item["response"] = pycon_assistant(item["query"])

# Save the sample data to a JSONL file
with open("pycon_test_data.jsonl", "w") as f:
    for item in pycon_test_data:
        f.write(json.dumps(item) + "\n")

# Display a few examples of the assistant's responses
for i in range(3):  # Show first 3 examples
    print(f"Query: {pycon_test_data[i]['query']}")
    print(f"Response: {pycon_test_data[i]['response']}")
    print("---")

## Create Custom Evaluators for PyCon Assistant

Now, let's create some custom evaluators specifically designed to evaluate a conference assistant:

In [ ]:
import re
from azure.ai.evaluation import (
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    FluencyEvaluator
)

# Initialize built-in evaluators
groundedness_eval = GroundednessEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)

# Create a simple custom evaluator to check if the response contains expected information
class ExpectedPatternEvaluator:
    def __init__(self):
        pass
    
    def __call__(self, *, response: str, expected_pattern: str, **kwargs):
        # If no pattern is provided, this test is not applicable
        if not expected_pattern:
            return {
                "value": None,
                "reason": "No expected pattern provided for this query."
            }
        
        # Check if the response contains any of the expected patterns (separated by |)
        patterns = expected_pattern.split('|')
        matches = [bool(re.search(pattern, response, re.IGNORECASE)) for pattern in patterns]
        match_ratio = sum(matches) / len(patterns) if patterns else 0
        
        return {
            "value": 1.0 if any(matches) else 0.0,  # Binary score: 1 if any pattern matches, 0 otherwise
            "match_ratio": match_ratio,  # Proportion of patterns that matched
            "matched_patterns": [patterns[i] for i, m in enumerate(matches) if m],
            "reason": f"Found {sum(matches)} of {len(patterns)} expected patterns in the response."
        }

# Create a custom evaluator for conference-specific tone and helpfulness
class ConferenceAssistantEvaluator:
    def __init__(self, model_config):
        self.model_config = model_config
        
    def __call__(self, *, query: str, response: str, category: str, **kwargs):
        # Use Azure OpenAI to evaluate the assistant's response
        try:
            eval_client = AzureOpenAI(
                azure_endpoint=self.model_config["azure_endpoint"],
                api_key=self.model_config["api_key"],
                api_version=self.model_config["api_version"]
            )
            
            # Create an evaluation prompt
            eval_prompt = f"""
            Please evaluate the following response from a PyCon conference assistant based on the criteria below:
            
            Question: {query}
            Response: {response}
            Question Category: {category}
            
            Evaluate on a scale of 1-5 (where 5 is best) for each:
            1. Helpfulness: Does it directly address the attendee's need?
            2. Friendliness: Is the tone welcoming and appropriate for a conference assistant?
            3. Conciseness: Is the answer appropriately brief while being complete?
            4. Accuracy: Based on common knowledge about tech conferences (not specific PyCon details).
            
            Provide your scores and a brief 1-2 sentence rationale for each.
            
            Format your response as a JSON object with the following structure:
            {{
                "helpfulness": {{"score": [1-5], "rationale": "your rationale"}},
                "friendliness": {{"score": [1-5], "rationale": "your rationale"}},
                "conciseness": {{"score": [1-5], "rationale": "your rationale"}},
                "accuracy": {{"score": [1-5], "rationale": "your rationale"}},
                "overall": {{"score": [1-5], "rationale": "your rationale"}}  
            }}
            """
            
            response = eval_client.chat.completions.create(
                model=self.model_config["deployment_name"],
                messages=[{"role": "user", "content": eval_prompt}],
                response_format={"type": "json_object"},
                temperature=0.1
            )
            
            # Parse and return the evaluation
            result = json.loads(response.choices[0].message.content)
            
            # Calculate average score across all dimensions
            scores = [result[key]["score"] for key in ["helpfulness", "friendliness", "conciseness", "accuracy"]]
            result["average_score"] = sum(scores) / len(scores)
            
            return result
            
        except Exception as e:
            print(f"Error in ConferenceAssistantEvaluator: {e}")
            return {
                "error": str(e),
                "helpfulness": {"score": 0, "rationale": "Evaluation failed"},
                "friendliness": {"score": 0, "rationale": "Evaluation failed"},
                "conciseness": {"score": 0, "rationale": "Evaluation failed"},
                "accuracy": {"score": 0, "rationale": "Evaluation failed"},
                "overall": {"score": 0, "rationale": "Evaluation failed"},
                "average_score": 0
            }

# Initialize our custom evaluators
expected_pattern_eval = ExpectedPatternEvaluator()
conference_assistant_eval = ConferenceAssistantEvaluator(model_config)

## Run Evaluation on the Test Dataset

Now let's evaluate our PyCon assistant using both built-in and custom evaluators:

In [ ]:
from azure.ai.evaluation import evaluate

# Run the evaluation on our test dataset
result = evaluate(
    data="pycon_test_data.jsonl",  # path to the data file
    evaluators={
        "groundedness": groundedness_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "expected_pattern": expected_pattern_eval,
        "conference_assistant": conference_assistant_eval
    },
    # Column mapping to tell the evaluator which columns to use
    evaluator_config={
        "groundedness": {
            "column_mapping": {
                "query": "${data.query}",
                "context": "${data.context}",
                "response": "${data.response}"
            }
        },
        "relevance": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}"
            }
        },
        "coherence": {
            "column_mapping": {
                "response": "${data.response}"
            }
        },
        "fluency": {
            "column_mapping": {
                "response": "${data.response}"
            }
        },
        "expected_pattern": {
            "column_mapping": {
                "response": "${data.response}",
                "expected_pattern": "${data.expected_pattern}"
            }
        },
        "conference_assistant": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}",
                "category": "${data.category}"
            }
        }
    },
    # Output path to save results
    output_path="./pycon_assistant_evaluation_results.json"
)

# Print evaluation results
print("Aggregate Metrics:")
print(json.dumps(result["metrics"], indent=2))
print("\nRow-level Results (first row):")
print(json.dumps(result["rows"][0], indent=2))

## Analyze the Evaluation Results

Let's analyze the evaluation results to identify strengths and areas for improvement:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the evaluation results
with open("pycon_assistant_evaluation_results.json", "r") as f:
    eval_results = json.load(f)

# Create a DataFrame from the row-level results
rows_data = []
for row in eval_results["rows"]:
    row_data = {
        "query": row["data"]["query"],
        "category": row["data"]["category"],
        "groundedness": row["outputs"]["groundedness"]["value"] if "groundedness" in row["outputs"] else None,
        "relevance": row["outputs"]["relevance"]["value"] if "relevance" in row["outputs"] else None,
        "coherence": row["outputs"]["coherence"]["value"] if "coherence" in row["outputs"] else None,
        "fluency": row["outputs"]["fluency"]["value"] if "fluency" in row["outputs"] else None,
        "expected_pattern": row["outputs"]["expected_pattern"]["value"] if "expected_pattern" in row["outputs"] else None,
    }
    
    # Add conference assistant specific scores if available
    if "conference_assistant" in row["outputs"]:
        conf_scores = row["outputs"]["conference_assistant"]
        if "helpfulness" in conf_scores:
            row_data["helpfulness"] = conf_scores["helpfulness"]["score"]
        if "friendliness" in conf_scores:
            row_data["friendliness"] = conf_scores["friendliness"]["score"]
        if "conciseness" in conf_scores:
            row_data["conciseness"] = conf_scores["conciseness"]["score"]
        if "accuracy" in conf_scores:
            row_data["accuracy"] = conf_scores["accuracy"]["score"]
        if "overall" in conf_scores:
            row_data["overall"] = conf_scores["overall"]["score"]
        if "average_score" in conf_scores:
            row_data["average_score"] = conf_scores["average_score"]
    
    rows_data.append(row_data)

df = pd.DataFrame(rows_data)

# Display the DataFrame
print("Evaluation Results by Query:")
display(df)

# Create visualizations to analyze results
plt.figure(figsize=(12, 6))

# Overall metrics by category
category_scores = df.groupby('category').mean()
category_scores.plot(kind='bar', figsize=(12, 6), title='Average Scores by Category')
plt.xticks(rotation=45)
plt.ylim(0, 5)  # Assuming scores are on a 0-5 scale
plt.tight_layout()
plt.show()

# Create radar charts for each category
categories = df['category'].unique()
metrics = ['groundedness', 'relevance', 'coherence', 'fluency', 'helpfulness', 'friendliness', 'conciseness', 'accuracy']
metrics = [m for m in metrics if m in df.columns]

# Function to create a radar chart
def create_radar_chart(category_data, title):
    # Number of variables
    N = len(metrics)
    
    # What will be the angle of each axis in the plot (divide the plot / number of variables)
    angles = [n / float(N) * 2 * 3.14159 for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Get the values
    values = category_data[metrics].mean().values.flatten().tolist()
    values += values[:1]  # Close the loop
    
    # Create the plot
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, polar=True)
    
    # Draw one axis per variable and add labels
    plt.xticks(angles[:-1], metrics, size=12)
    
    # Draw the chart
    ax.plot(angles, values, linewidth=2, linestyle='solid')
    ax.fill(angles, values, alpha=0.1)
    
    # Set y-axis limits
    plt.ylim(0, 5)
    
    # Add title
    plt.title(title, size=15)
    
    return fig

# Create radar charts for overall and each category
create_radar_chart(df, 'Overall Performance')
plt.tight_layout()
plt.show()

for category in categories:
    category_data = df[df['category'] == category]
    create_radar_chart(category_data, f'Performance: {category} questions')
    plt.tight_layout()
    plt.show()

## Generate Insights and Recommendations

Based on the evaluation results, let's generate insights and recommendations for improving the PyCon assistant:

In [ ]:
# Generate insights using Azure OpenAI
def generate_insights(eval_results, metrics_df):
    try:
        # Format the data for analysis
        metrics_str = metrics_df.to_string()
        aggregate = json.dumps(eval_results["metrics"], indent=2)
        
        insight_prompt = f"""
        You are an expert in analyzing AI assistant evaluation results. Please analyze the following evaluation 
        results for a PyCon conference assistant and provide insights and recommendations.
        
        Evaluation Metrics by Query:
        {metrics_str}
        
        Aggregate Metrics:
        {aggregate}
        
        Please provide:
        1. Key strengths (3-5 bullet points)
        2. Areas for improvement (3-5 bullet points)
        3. Specific recommendations to enhance the assistant's performance
        4. Any patterns you notice across different question categories
        
        Format your response in Markdown with clear sections and bullet points.
        """
        
        response = client.chat.completions.create(
            model=model_config["deployment_name"],
            messages=[{"role": "user", "content": insight_prompt}],
            temperature=0.3,
            max_tokens=1000
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating insights: {e}")
        return "Error generating insights. Please check the evaluation results manually."

# Generate and display insights
insights = generate_insights(eval_results, df)
print(insights)

## Conclusion

This notebook demonstrates how to evaluate a custom PyCon assistant copilot using the Azure AI Evaluation SDK. We've shown how to:

1. Define a PyCon-specific system prompt to create a specialized assistant
2. Create a test dataset with conference-specific questions
3. Use built-in evaluators to assess basic quality metrics (groundedness, relevance, etc.)
4. Create custom evaluators specifically designed for conference assistants
5. Run comprehensive evaluations across different question categories
6. Analyze results to identify strengths and areas for improvement
7. Generate insights and recommendations for enhancing the assistant

This evaluation framework can be extended to evaluate other types of specialized assistants by:
- Modifying the system prompt for different domains
- Creating domain-specific test datasets
- Adding custom evaluators tailored to domain requirements
- Adjusting analysis to focus on domain-specific performance metrics

The Azure AI Evaluation SDK provides a powerful and flexible framework for evaluating and improving AI assistants across various specialized domains.